In [1]:
using CSV
using DataFrames
using Dates

In [2]:
ENV["COLUMNS"] = 1000;

In [3]:
rawdata = DataFrame(CSV.File("../rawdata/output_HHS_US_2021-01-16_one_column.csv"))

,Column1,hospital_name,ccn,collection_week,fips_code,hospitalization_percentage,ICU_percentages,hospitalization_2020-12-22,target_date
,Int64,String,String?,Date,Float64,Float64?,Float64?,Float64?,Date
1,10,PRATTVILLE BAPTIST HOSPITAL,010108,2020-12-11,1001.0,1.0,0.122066,3.69904,2020-12-22
2,11,THOMAS HOSPITAL,010100,2020-12-11,1003.0,0.522534,0.16055,5.18507,2020-12-22
3,12,SOUTH BALDWIN REGIONAL MEDICAL CENTER,010083,2020-12-11,1003.0,0.314044,0.444444,3.11624,2020-12-22
4,13,NORTH BALDWIN INFIRMARY,010129,2020-12-11,1003.0,0.163422,0.104167,1.62163,2020-12-22
5,14,MEDICAL CENTER BARBOUR,010069,2020-12-11,1005.0,1.0,0.178571,0.640395,2020-12-22
6,15,BIBB MEDICAL CENTER,010058,2020-12-11,1007.0,0.9,missing,1.39263,2020-12-22
7,16,ST VINCENTS BLOUNT,011305,2020-12-11,1009.0,0.95,0.25641,3.90948,2020-12-22
8,17,BULLOCK COUNTY HOSPITAL,010110,2020-12-11,1011.0,0.0,missing,0.0,2020-12-22
9,18,REGIONAL MEDICAL CENTER OF CENTRAL ALABAMA,010150,2020-12-11,1013.0,0.95,0.322581,0.709286,2020-12-22


In [4]:
data = select(rawdata,
    :hospital_name => ByRow(titlecase) => :hospital,
    :ccn => :hospital_id,
    :target_date => :date,
    ["hospitalization_2020-12-22", "ICU_percentages"] => ByRow((x,y) -> x*coalesce(y,0.3)) => :admitted_moderate_icu,
    ["hospitalization_2020-12-22", "ICU_percentages"] => ByRow((x,y) -> x*(1.0-coalesce(y,0.3))) => :admitted_moderate_acute,
    "hospitalization_2020-12-22" => :admitted_moderate_allbeds,
)
sort!(data, [:hospital, :hospital_id, :date])

,hospital,hospital_id,date,admitted_moderate_icu,admitted_moderate_acute,admitted_moderate_allbeds
,String,String?,Date,Float64?,Float64?,Float64?
1,Abbeville Area Medical Center,421301,2020-12-22,0.0,0.537053,0.537053
2,Abbeville Area Medical Center,missing,2020-12-23,0.0,0.555188,0.555188
3,Abbeville Area Medical Center,missing,2020-12-24,0.0,0.568869,0.568869
4,Abbeville Area Medical Center,missing,2020-12-25,0.0,0.576186,0.576186
5,Abbeville Area Medical Center,missing,2020-12-26,0.0,0.608002,0.608002
6,Abbeville Area Medical Center,missing,2020-12-27,0.0,0.577841,0.577841
7,Abbeville Area Medical Center,missing,2020-12-28,0.0,0.560201,0.560201
8,Abbeville Area Medical Center,missing,2020-12-29,0.0,0.559289,0.559289
9,Abbeville Area Medical Center,missing,2020-12-30,0.0,0.561722,0.561722


In [5]:
data |> CSV.write("../data/hhs_forecast_2020_12_26.csv")

"../data/hhs_forecast_2020_12_26.csv"